# Manipulating the Pandas DataFrame

The Jupyter notebook for this demo can be found in:
   - docs/quick_start/demo/op2_pandas_unstack.ipynb
   - https://github.com/SteveDoyle2/pyNastran/tree/master/docs/quick_start/demo/op2_pandas_unstack.ipynb
   
### This example will use pandas unstack
The unstack method on a DataFrame moves on index level from rows to columns.  First let's read in some data:

In [1]:
import os
import pyNastran
pkg_path = pyNastran.__path__[0]
from pyNastran.op2.op2 import read_op2
import pandas as pd
pd.set_option('display.precision', 2)

op2_filename = os.path.join(pkg_path, '..', 'models', 'iSat', 'iSat_launch_100Hz.op2')
from pyNastran.op2.op2 import read_op2
isat = read_op2(op2_filename, build_dataframe=True, debug=False, skip_undefined_matrices=True)

self.cannot apply column_names=['Mode', 'Freq'] to RealStrainEnergyArray: 'QUAD4'
self.cannot apply column_names=['Mode', 'Freq'] to RealStrainEnergyArray: 'TRIA3'
self.cannot apply column_names=['Mode', 'Freq'] to RealStrainEnergyArray: 'HEXA'
self.cannot apply column_names=['Mode', 'Freq'] to RealStrainEnergyArray: 'BAR'
self.cannot apply column_names=['Mode', 'Freq'] to RealStrainEnergyArray: 'BUSH'


In [2]:
cbar = isat.cbar_force[1].data_frame

In [3]:
cbar.head()

Mode                               1         2         3         4         5   \
Freq                            8.36      9.51      15.67     20.24     20.31   
Eigenvalue                  2758.15   3568.63   9689.98   16168.04  16278.16    
Radians                        52.52     59.74     98.44     127.15    127.59   
ElementID Item                                                                  
3323      bending_moment_a1     -0.16     -0.23     -1.33 -2.32e+00     -1.88   
          bending_moment_a2      0.19      0.05      0.18  5.58e-03     -0.11   
          bending_moment_b1      0.17      0.21      2.01  2.66e+00      1.88   
          bending_moment_b2     -0.19     -0.05     -0.18 -3.54e-03      0.11   
          shear1                -0.13     -0.18     -1.33 -1.99e+00     -1.50   

Mode                               6         7         8         9         10  \
Freq                            20.55     21.50     21.71     21.72     28.54   
Eigenvalue                  16679.71  18248.43  18600.70  18632.55  32159.89    
Radians                        129.15    135.09    136.38    136.50    179.33   
ElementID Item                                                                  
3323      bending_moment_a1     -0.80 -1.34e-03      1.42      1.47      4.65   
          bending_moment_a2     -0.42 -4.19e-03     -1.11      0.10     -1.57   
          bending_moment_b1      0.73  2.29e-03     -1.38     -1.31     -3.98   
          bending_moment_b2      0.43  4.18e-03      1.11     -0.10      1.57   
          shear1                -0.61 -1.45e-03      1.12      1.11      3.45   

Mode                         ...        24        25        26        27  \
Freq                         ...     80.08     86.49     88.17     88.48   
Eigenvalue                   ... 253141.17 295300.94 306886.00 309040.66   
Radians                      ...    503.13    543.42    553.97    555.91   
ElementID Item               ...                                           
3323      bending_moment_a1  ...    -43.42     63.36    -43.07     -3.35   
          bending_moment_a2  ...     -4.50      5.33      1.63      4.86   
          bending_moment_b1  ...     34.70    -74.02     35.13      3.54   
          bending_moment_b2  ...      4.50     -5.34     -1.62     -4.86   
          shear1             ...    -31.25     54.95    -31.28     -2.76   

Mode                               28        29        30        31        32  \
Freq                            89.93     94.29     94.37     96.04     98.70   
Eigenvalue                  319267.09 350984.50 351566.19 364166.31 384601.34   
Radians                        565.04    592.44    592.93    603.46    620.16   
ElementID Item                                                                  
3323      bending_moment_a1     11.08    -14.38      0.75     29.36      0.49   
          bending_moment_a2      2.15      0.09     -1.27    -10.58     -0.67   
          bending_moment_b1    -15.03     10.97     -0.67    -17.69     -0.63   
          bending_moment_b2     -2.15     -0.08      1.27     10.56      0.67   
          shear1                10.44    -10.14      0.57     18.82      0.44   

Mode                               33  
Freq                            98.89  
Eigenvalue                  386090.47  
Radians                        621.36  
ElementID Item                         
3323      bending_moment_a1     -4.56  
          bending_moment_a2      3.48  
          bending_moment_b1      6.39  
          bending_moment_b2     -3.49  
          shear1                -4.38  

[5 rows x 33 columns]

First I'm going to pull out a small subset to work with

In [4]:
csub = cbar.loc[3323:3324,1:2]
csub

Mode                              1       2
Freq                           8.36    9.51
Eigenvalue                  2758.15 3568.63
Radians                       52.52   59.74
ElementID Item                             
3323      bending_moment_a1   -0.16   -0.23
          bending_moment_a2    0.19    0.05
          bending_moment_b1    0.17    0.21
          bending_moment_b2   -0.19   -0.05
          shear1              -0.13   -0.18
          shear2               0.15    0.04
          axial                0.80   -0.21
          torque              -0.04    0.06
3324      bending_moment_a1    0.14    0.29
          bending_moment_a2   -0.19   -0.05
          bending_moment_b1   -0.15   -0.26
          bending_moment_b2    0.19    0.05
          shear1               0.12    0.22
          shear2              -0.15   -0.04
          axial               -0.80    0.21
          torque               0.04   -0.06

I happen to like the way that's organized, but let's say that I want the have the item descriptions in columns and the mode ID's and element numbers in rows.  To do that, I'll first move the element ID's up to the columns using a .unstack(level=0) and the transpose the result:

In [5]:
csub.unstack(level=0).T

Item                                    axial  bending_moment_a1  \
Mode Freq Eigenvalue Radians ElementID                             
1    8.36 2758.15    52.52   3323        0.80              -0.16   
                             3324       -0.80               0.14   
2    9.51 3568.63    59.74   3323       -0.21              -0.23   
                             3324        0.21               0.29   

Item                                    bending_moment_a2  bending_moment_b1  \
Mode Freq Eigenvalue Radians ElementID                                         
1    8.36 2758.15    52.52   3323                    0.19               0.17   
                             3324                   -0.19              -0.15   
2    9.51 3568.63    59.74   3323                    0.05               0.21   
                             3324                   -0.05              -0.26   

Item                                    bending_moment_b2  shear1  shear2  \
Mode Freq Eigenvalue Radians ElementID                                      
1    8.36 2758.15    52.52   3323                   -0.19   -0.13    0.15   
                             3324                    0.19    0.12   -0.15   
2    9.51 3568.63    59.74   3323                   -0.05   -0.18    0.04   
                             3324                    0.05    0.22   -0.04   

Item                                    torque  
Mode Freq Eigenvalue Radians ElementID          
1    8.36 2758.15    52.52   3323        -0.04  
                             3324         0.04  
2    9.51 3568.63    59.74   3323         0.06  
                             3324        -0.06

unstack requires unique row indices so I can't work with CQUAD4 stresses as they're currently output, but I'll work with CHEXA stresses.  Let's pull out the first two elements and first two modes:

In [6]:
chs = isat.chexa_stress[1].data_frame.loc[3684:3685,1:2]
chs

Mode                               1         2
Freq                            8.36      9.51
Eigenvalue                   2758.15   3568.63
Radians                        52.52     59.74
ElementID NodeID Item                         
3684      0      oxx        1.22e-12 -3.41e-13
                 oyy       -3.35e-12 -2.27e-13
                 ozz        1.25e-12  4.55e-13
                 txy       -3.27e-13  1.82e-12
                 tyz        2.84e-14  3.98e-13
...                              ...       ...
3685      1037   txz       -2.84e-13 -1.82e-12
                 omax      -7.47e-15  2.08e-12
                 omid      -1.15e-13 -2.71e-13
                 omin      -1.00e-12 -1.70e-12
                 von_mises  9.43e-13  3.30e-12

[180 rows x 2 columns]

Now I want to put ElementID and the Node ID in the rows along with the Load ID, and have the items in the columns:

In [7]:
cht = chs.unstack(level=[0,1]).T
cht

Item                                               omax      omid      omin  \
Mode Freq Eigenvalue Radians ElementID NodeID                                 
1    8.36 2758.15    52.52   3684      0       1.48e-12  1.02e-12 -3.38e-12   
                                       55      4.81e-12  1.92e-13 -3.57e-13   
                                       51      2.32e-12  1.49e-13 -1.41e-12   
                                       778    -1.38e-12 -3.27e-12 -6.12e-12   
                                       758     5.79e-12  4.11e-12  7.57e-14   
...                                                 ...       ...       ...   
2    9.51 3568.63    59.74   3685      1015    1.19e-12 -9.78e-14 -6.96e-13   
                                       50      4.88e-13 -1.28e-13 -1.21e-12   
                                       46      6.22e-13  2.26e-14 -5.87e-13   
                                       1031    2.32e-12 -6.90e-13 -1.63e-12   
                                       1037    2.08e-12 -2.71e-13 -1.70e-12   

Item                                                oxx       oyy       ozz  \
Mode Freq Eigenvalue Radians ElementID NodeID                                 
1    8.36 2758.15    52.52   3684      0       1.22e-12 -3.35e-12  1.25e-12   
                                       55      4.53e-12 -2.42e-13  3.55e-13   
                                       51     -1.39e-12  2.32e-12  1.35e-13   
                                       778    -6.08e-12 -1.38e-12 -3.31e-12   
                                       758     5.68e-12  1.14e-13  4.18e-12   
...                                                 ...       ...       ...   
2    9.51 3568.63    59.74   3685      1015    3.41e-13 -5.68e-14  1.14e-13   
                                       50     -3.98e-13  1.14e-13 -5.68e-13   
                                       46     -2.84e-13  3.41e-13  0.00e+00   
                                       1031   -2.27e-13 -6.82e-13  9.09e-13   
                                       1037    4.55e-13 -3.41e-13  0.00e+00   

Item                                                txy       txz       tyz  \
Mode Freq Eigenvalue Radians ElementID NodeID                                 
1    8.36 2758.15    52.52   3684      0      -3.27e-13 -2.27e-13  2.84e-14   
                                       55     -6.54e-13 -9.30e-13  2.20e-15   
                                       51     -1.30e-13 -1.46e-13  7.51e-15   
                                       778    -5.81e-14 -3.41e-13 -1.97e-14   
                                       758    -4.55e-13 -3.41e-13 -3.91e-14   
...                                                 ...       ...       ...   
2    9.51 3568.63    59.74   3685      1015   -2.27e-13 -9.09e-13  1.13e-13   
                                       50     -5.68e-13 -4.90e-13 -2.27e-13   
                                       46      4.54e-13  9.59e-14 -2.27e-13   
                                       1031    4.55e-13 -1.82e-12 -2.27e-13   
                                       1037   -3.98e-13 -1.82e-12 -1.14e-13   

Item                                           von_mises  
Mode Freq Eigenvalue Radians ElementID NodeID             
1    8.36 2758.15    52.52   3684      0        4.64e-12  
                                       55       4.92e-12  
                                       51       3.25e-12  
                                       778      4.14e-12  
                                       758      5.09e-12  
...                                                  ...  
2    9.51 3568.63    59.74   3685      1015     1.67e-12  
                                       50       1.49e-12  
                                       46       1.05e-12  
                                       1031     3.57e-12  
                                       1037     3.30e-12  

[68 rows x 10 columns]

Maybe I'd like my rows organized with the modes on the inside.  I can do that by swapping levels:

We actually need to get rid of the extra rows using dropna():

In [8]:
cht = cht.dropna()
cht

Item                                               omax      omid      omin  \
Mode Freq Eigenvalue Radians ElementID NodeID                                 
1    8.36 2758.15    52.52   3684      0       1.48e-12  1.02e-12 -3.38e-12   
                                       55      4.81e-12  1.92e-13 -3.57e-13   
                                       51      2.32e-12  1.49e-13 -1.41e-12   
                                       778    -1.38e-12 -3.27e-12 -6.12e-12   
                                       758     5.79e-12  4.11e-12  7.57e-14   
                                       60      2.88e-12  1.71e-12 -4.27e-12   
                                       56      1.66e-12 -1.65e-12 -5.92e-12   
                                       880     2.63e-12  2.11e-12 -4.74e-12   
                                       952    -8.35e-15 -1.79e-12 -3.00e-12   
                             3685      0       5.11e-13  1.44e-13 -5.41e-13   
                                       45      8.09e-13 -5.47e-13 -8.09e-13   
                                       41      7.67e-13 -1.49e-13 -4.37e-13   
                                       1021    7.75e-13  1.86e-13 -5.38e-13   
                                       1015    8.67e-13  1.56e-13 -6.65e-13   
                                       50      9.64e-13  7.11e-13 -6.57e-14   
                                       46      2.26e-12  7.91e-13  7.55e-14   
                                       1031   -8.72e-13 -1.24e-12 -1.87e-12   
                                       1037   -7.47e-15 -1.15e-13 -1.00e-12   
2    9.51 3568.63    59.74   3684      0       2.22e-12 -1.06e-13 -2.23e-12   
                                       55      3.64e-12  9.55e-13 -1.33e-12   
                                       51      7.48e-13 -3.94e-13 -1.46e-12   
                                       778     1.24e-12 -5.76e-13 -2.37e-12   
                                       758     1.03e-12 -1.04e-12 -3.57e-12   
                                       60      1.02e-12 -2.20e-12 -2.85e-12   
                                       56      5.04e-13 -3.52e-13 -2.37e-12   
                                       880     1.79e-12  6.09e-13 -2.51e-12   
                                       952     1.30e-12  1.93e-13 -2.63e-12   
                             3685      0       1.18e-12 -3.91e-15 -9.47e-13   
                                       45      8.96e-13 -2.21e-13 -1.07e-12   
                                       41      8.01e-13 -3.97e-13 -8.59e-13   
                                       1021    9.04e-13  4.58e-13 -3.01e-12   
                                       1015    1.19e-12 -9.78e-14 -6.96e-13   
                                       50      4.88e-13 -1.28e-13 -1.21e-12   
                                       46      6.22e-13  2.26e-14 -5.87e-13   
                                       1031    2.32e-12 -6.90e-13 -1.63e-12   
                                       1037    2.08e-12 -2.71e-13 -1.70e-12   

Item                                                oxx       oyy       ozz  \
Mode Freq Eigenvalue Radians ElementID NodeID                                 
1    8.36 2758.15    52.52   3684      0       1.22e-12 -3.35e-12  1.25e-12   
                                       55      4.53e-12 -2.42e-13  3.55e-13   
                                       51     -1.39e-12  2.32e-12  1.35e-13   
                                       778    -6.08e-12 -1.38e-12 -3.31e-12   
                                       758     5.68e-12  1.14e-13  4.18e-12   
                                       60      2.63e-12  1.90e-12 -4.21e-12   
                                       56     -5.87e-12  1.65e-12 -1.69e-12   
                                       880    -4.72e-12  2.10e-12  2.61e-12   
                                       952    -1.73e-12 -1.14e-13 -2.96e-12   
                             3685      0       4.97e-13  1.56e-13 -5.40e-13   
                                       45      6.

In [9]:
# mode, eigr, freq, rad, eids, nids # initial
# nids, eids, eigr, freq, rad, mode # final

cht.swaplevel(0,4).swaplevel(1,5).swaplevel(2,5).swaplevel(4, 5)

Item                                               omax      omid      omin  \
ElementID NodeID Freq Radians Eigenvalue Mode                                 
3684      0      8.36 52.52   2758.15    1     1.48e-12  1.02e-12 -3.38e-12   
          55     8.36 52.52   2758.15    1     4.81e-12  1.92e-13 -3.57e-13   
          51     8.36 52.52   2758.15    1     2.32e-12  1.49e-13 -1.41e-12   
          778    8.36 52.52   2758.15    1    -1.38e-12 -3.27e-12 -6.12e-12   
          758    8.36 52.52   2758.15    1     5.79e-12  4.11e-12  7.57e-14   
          60     8.36 52.52   2758.15    1     2.88e-12  1.71e-12 -4.27e-12   
          56     8.36 52.52   2758.15    1     1.66e-12 -1.65e-12 -5.92e-12   
          880    8.36 52.52   2758.15    1     2.63e-12  2.11e-12 -4.74e-12   
          952    8.36 52.52   2758.15    1    -8.35e-15 -1.79e-12 -3.00e-12   
3685      0      8.36 52.52   2758.15    1     5.11e-13  1.44e-13 -5.41e-13   
          45     8.36 52.52   2758.15    1     8.09e-13 -5.47e-13 -8.09e-13   
          41     8.36 52.52   2758.15    1     7.67e-13 -1.49e-13 -4.37e-13   
          1021   8.36 52.52   2758.15    1     7.75e-13  1.86e-13 -5.38e-13   
          1015   8.36 52.52   2758.15    1     8.67e-13  1.56e-13 -6.65e-13   
          50     8.36 52.52   2758.15    1     9.64e-13  7.11e-13 -6.57e-14   
          46     8.36 52.52   2758.15    1     2.26e-12  7.91e-13  7.55e-14   
          1031   8.36 52.52   2758.15    1    -8.72e-13 -1.24e-12 -1.87e-12   
          1037   8.36 52.52   2758.15    1    -7.47e-15 -1.15e-13 -1.00e-12   
3684      0      9.51 59.74   3568.63    2     2.22e-12 -1.06e-13 -2.23e-12   
          55     9.51 59.74   3568.63    2     3.64e-12  9.55e-13 -1.33e-12   
          51     9.51 59.74   3568.63    2     7.48e-13 -3.94e-13 -1.46e-12   
          778    9.51 59.74   3568.63    2     1.24e-12 -5.76e-13 -2.37e-12   
          758    9.51 59.74   3568.63    2     1.03e-12 -1.04e-12 -3.57e-12   
          60     9.51 59.74   3568.63    2     1.02e-12 -2.20e-12 -2.85e-12   
          56     9.51 59.74   3568.63    2     5.04e-13 -3.52e-13 -2.37e-12   
          880    9.51 59.74   3568.63    2     1.79e-12  6.09e-13 -2.51e-12   
          952    9.51 59.74   3568.63    2     1.30e-12  1.93e-13 -2.63e-12   
3685      0      9.51 59.74   3568.63    2     1.18e-12 -3.91e-15 -9.47e-13   
          45     9.51 59.74   3568.63    2     8.96e-13 -2.21e-13 -1.07e-12   
          41     9.51 59.74   3568.63    2     8.01e-13 -3.97e-13 -8.59e-13   
          1021   9.51 59.74   3568.63    2     9.04e-13  4.58e-13 -3.01e-12   
          1015   9.51 59.74   3568.63    2     1.19e-12 -9.78e-14 -6.96e-13   
          50     9.51 59.74   3568.63    2     4.88e-13 -1.28e-13 -1.21e-12   
          46     9.51 59.74   3568.63    2     6.22e-13  2.26e-14 -5.87e-13   
          1031   9.51 59.74   3568.63    2     2.32e-12 -6.90e-13 -1.63e-12   
          1037   9.51 59.74   3568.63    2     2.08e-12 -2.71e-13 -1.70e-12   

Item                                                oxx       oyy       ozz  \
ElementID NodeID Freq Radians Eigenvalue Mode                                 
3684      0      8.36 52.52   2758.15    1     1.22e-12 -3.35e-12  1.25e-12   
          55     8.36 52.52   2758.15    1     4.53e-12 -2.42e-13  3.55e-13   
          51     8.36 52.52   2758.15    1    -1.39e-12  2.32e-12  1.35e-13   
          778    8.36 52.52   2758.15    1    -6.08e-12 -1.38e-12 -3.31e-12   
          758    8.36 52.52   2758.15    1     5.68e-12  1.14e-13  4.18e-12   
          60     8.36 52.52   2758.15    1     2.63e-12  1.90e-12 -4.21e-12   
          56     8.36 52.52   2758.15    1    -5.87e-12  1.65e-12 -1.69e-12   
          880    8.36 52.52   2758.15    1    -4.72e-12  2.10e-12  2.61e-12   
          952    8.36 52.52   2758.15    1    -1.73e-12 -1.14e-13 -2.96e-12   
3685      0      8.36 52.52   2758.15    1     4.97e-13  1.56e-13 -5.40e-13   
          45     8.36 52.52   2758.15    1     6.

Alternatively I can do that by first using reset_index to move all the index columns into data, and then using set_index to define the order of columns I want as my index:

In [10]:
cht.reset_index().set_index(['ElementID','NodeID','Mode','Freq']).sort_index()

Item                        Eigenvalue  Radians      omax      omid      omin  \
ElementID NodeID Mode Freq                                                      
3684      0      1    8.36     2758.15    52.52  1.48e-12  1.02e-12 -3.38e-12   
                 2    9.51     3568.63    59.74  2.22e-12 -1.06e-13 -2.23e-12   
          51     1    8.36     2758.15    52.52  2.32e-12  1.49e-13 -1.41e-12   
                 2    9.51     3568.63    59.74  7.48e-13 -3.94e-13 -1.46e-12   
          55     1    8.36     2758.15    52.52  4.81e-12  1.92e-13 -3.57e-13   
                 2    9.51     3568.63    59.74  3.64e-12  9.55e-13 -1.33e-12   
          56     1    8.36     2758.15    52.52  1.66e-12 -1.65e-12 -5.92e-12   
                 2    9.51     3568.63    59.74  5.04e-13 -3.52e-13 -2.37e-12   
          60     1    8.36     2758.15    52.52  2.88e-12  1.71e-12 -4.27e-12   
                 2    9.51     3568.63    59.74  1.02e-12 -2.20e-12 -2.85e-12   
          758    1    8.36     2758.15    52.52  5.79e-12  4.11e-12  7.57e-14   
                 2    9.51     3568.63    59.74  1.03e-12 -1.04e-12 -3.57e-12   
          778    1    8.36     2758.15    52.52 -1.38e-12 -3.27e-12 -6.12e-12   
                 2    9.51     3568.63    59.74  1.24e-12 -5.76e-13 -2.37e-12   
          880    1    8.36     2758.15    52.52  2.63e-12  2.11e-12 -4.74e-12   
                 2    9.51     3568.63    59.74  1.79e-12  6.09e-13 -2.51e-12   
          952    1    8.36     2758.15    52.52 -8.35e-15 -1.79e-12 -3.00e-12   
                 2    9.51     3568.63    59.74  1.30e-12  1.93e-13 -2.63e-12   
3685      0      1    8.36     2758.15    52.52  5.11e-13  1.44e-13 -5.41e-13   
                 2    9.51     3568.63    59.74  1.18e-12 -3.91e-15 -9.47e-13   
          41     1    8.36     2758.15    52.52  7.67e-13 -1.49e-13 -4.37e-13   
                 2    9.51     3568.63    59.74  8.01e-13 -3.97e-13 -8.59e-13   
          45     1    8.36     2758.15    52.52  8.09e-13 -5.47e-13 -8.09e-13   
                 2    9.51     3568.63    59.74  8.96e-13 -2.21e-13 -1.07e-12   
          46     1    8.36     2758.15    52.52  2.26e-12  7.91e-13  7.55e-14   
                 2    9.51     3568.63    59.74  6.22e-13  2.26e-14 -5.87e-13   
          50     1    8.36     2758.15    52.52  9.64e-13  7.11e-13 -6.57e-14   
                 2    9.51     3568.63    59.74  4.88e-13 -1.28e-13 -1.21e-12   
          1015   1    8.36     2758.15    52.52  8.67e-13  1.56e-13 -6.65e-13   
                 2    9.51     3568.63    59.74  1.19e-12 -9.78e-14 -6.96e-13   
          1021   1    8.36     2758.15    52.52  7.75e-13  1.86e-13 -5.38e-13   
                 2    9.51     3568.63    59.74  9.04e-13  4.58e-13 -3.01e-12   
          1031   1    8.36     2758.15    52.52 -8.72e-13 -1.24e-12 -1.87e-12   
                 2    9.51     3568.63    59.74  2.32e-12 -6.90e-13 -1.63e-12   
          1037   1    8.36     2758.15    52.52 -7.47e-15 -1.15e-13 -1.00e-12   
                 2    9.51     3568.63    59.74  2.08e-12 -2.71e-13 -1.70e-12   

Item                             oxx       oyy       ozz       txy       txz  \
ElementID NodeID Mode Freq                                                     
3684      0      1    8.36  1.22e-12 -3.35e-12  1.25e-12 -3.27e-13 -2.27e-13   
                 2    9.51 -3.41e-13 -2.27e-13  4.55e-13  1.82e-12  1.14e-12   
          51     1    8.36 -1.39e-12  2.32e-12  1.35e-13 -1.30e-13 -1.46e-13   
                 2    9.51 -3.41e-13  5.68e-14 -8.24e-13  1.57e-13  3.85e-13   
          55     1    8.36  4.53e-12 -2.42e-13  3.55e-13 -6.54e-13 -9.30e-13   
                 2    9.51  2.33e-12 -5.68e-14  9.95e-13  1.82e-12  4.26e-13   
          56     1    8.36 -5.87e-12  1.65e-12 -1.69e-12 -3.07e-13 -4.22e-13   
                 2    9.51 -1.82e-12 -7.96e-13  3.98e-13 -9.10e-13 -3.23e-13   
          60     1    8.36  2.63e-12  1.90e-12 -4.21e-12 -4.26e-13 -6.34e-13   
                 2    9.51 -1.48e-12 -3.4